In [1]:
from collections import Counter
import matplotlib.pyplot as plt
from pysankey2 import Sankey
from scipy import sparse
from scipy.sparse import hstack, vstack, coo_matrix
from scipy.io import mmread
from sklearn import metrics
import math
import time
import anndata
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import random
import os
from warnings import filterwarnings
import scipy.sparse as sp
from operator import itemgetter
filterwarnings("ignore")
import json, os
import math, copy, time
import numpy as np
from collections import defaultdict
import pyHGT2
from pyHGT2.data import *
from pyHGT2.utils import *
from pyHGT2.model1 import *
from pyHGT2.conv import *
from sklearn import metrics

from collections import Counter
import matplotlib.pyplot as plt
from pysankey2 import Sankey
from scipy.sparse import hstack, vstack, coo_matrix
from scipy.io import mmread
from sklearn import metrics
import math
import time
import anndata
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import random
import os
from scipy import sparse
from scipy.io import mmread
from scipy.sparse import hstack, vstack, coo_matrix
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import normalized_mutual_info_score

import xlwt
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
def sub_anndatas(index,ad1,dim1,ad2,dim2,ad3):
    def sub_ad(ad,id,dim):
        if dim == 0:
            ad = ad[id,:]
        elif dim == 1:
            ad = ad[:,id]
        return(ad)

    # sub sample
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)

    # delete feature1 and feature2 not in sample
    index1 = np.where(np.array(ad1.X.sum(dim1)).squeeze()>0)[0]
    index2 = np.where(np.array(ad2.X.sum(dim2)).squeeze()>0)[0]
    ad3_new = sub_ad(sub_ad(ad3,index1,0),index2,1)

    # delete feature1 and feature2 not link
    index1 = index1[np.where(np.array(ad3_new.X.sum(1)).squeeze()>0)[0]]
    index2 = index2[np.where(np.array(ad3_new.X.sum(0)).squeeze()>0)[0]]

    ad1 = sub_ad(ad1,index1,1-dim1)
    ad2 = sub_ad(ad2,index2,1-dim2)
    ad3 = sub_ad(sub_ad(ad3,index1,0),index2,1)

    #delete sample don't in features
    index = np.where((np.array(ad1.X.sum(1-dim1)).squeeze()>0) & (np.array(ad2.X.sum(1-dim2)).squeeze()>0))[0]
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)
    return (ad1,ad2,ad3,index1,index2,index)

## 读取cell_line数据集

In [3]:
ds = 'celline'
os.chdir('/fs/ess/PCON0022/LiJingxian/Data/ATAC/mult/celline/')
gene_peak = anndata.read_mtx('Gene_Peak.mtx')
gene_cell = anndata.read_mtx('Gene_Cell.mtx')
peak_cell = anndata.read_mtx('Peak_Cell.mtx')
gene_names = pd.read_csv('Gene_names.tsv', sep='\t', header=None)
cell_names = pd.read_csv('Cell_names.tsv', sep='\t', header=None)
peak_names = pd.read_csv('Peak_names.tsv', sep='\t', header=None)
peak_cell.obs_names = peak_names[0]
peak_cell.var_names = cell_names[0]
gene_cell.obs_names = gene_names[0]
gene_cell.var_names = cell_names[0]
gene_peak.obs_names = gene_names[0]
gene_peak.var_names = peak_names[0]

In [4]:
cell_names = pd.Series(list(cell_names.iloc[:,0])).map(lambda x:x.split('_')[1])
true_label = pd.DataFrame(list(cell_names),columns=['cell_type'])
cell_type = true_label
cell_type

,cell_type
0,HCT116
1,HCT116
2,HCT116
3,HCT116
4,HCT116
...,...
544,PDX2
545,PDX2
546,PDX2
547,PDX2


In [5]:
cell_type['number'] = 1
cell_count = cell_type.groupby('cell_type').count()
cell_count =cell_count.sort_values(by='number',ascending=True, inplace=False)
rare_name = cell_count.iloc[0:2].index
ordinary_name = cell_count.iloc[-2:].index

In [6]:
cell_count

,number
cell_type,
HeLa.S3,42
K562,74
HCT116,90
PDX2,167
PDX1,176


In [7]:
rare_name

Index(['HeLa.S3', 'K562'], dtype='object', name='cell_type')

In [8]:
ordinary_name

Index(['PDX2', 'PDX1'], dtype='object', name='cell_type')

In [9]:
ordinary_index = list(cell_type[np.array(cell_type['cell_type']==ordinary_name[0])+np.array(cell_type['cell_type']==ordinary_name[1])].index)
one_rare_index = list(cell_type[np.array(cell_type['cell_type']==rare_name[0])].index)
# two_rare_index = list(cell_type[np.array(cell_type['cell_type']==rare_name[0])+np.array(cell_type['cell_type']==rare_name[1])].index)
two_rare_index_1 = list(cell_type[np.array(cell_type['cell_type']==rare_name[0])].index)
two_rare_index_2 = list(cell_type[np.array(cell_type['cell_type']==rare_name[1])].index)

In [19]:
len(ordinary_index)

343

In [18]:
len(one_rare_index)

42

In [16]:
len(two_rare_index_1)

42

In [17]:
len(two_rare_index_2)

74

## Ordinary1 Rare1

In [21]:
true_label = np.array(cell_names)
ip_file = '/fs/ess/scratch/PCON0022/mtduan/cell_line/'
filetype_name = 'Ordinary1_Rare1'

for file_num in tqdm(range(50)):
    l = []
    l = l + random.sample(ordinary_index,290)
    l = l + random.sample(one_rare_index,10)
    l = np.sort(np.array(l))
    label = true_label[l]
#     print(l)
#     print(label)
    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X

    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    ## python_input
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label300.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [09:53<00:00, 11.88s/it]


## Ordinary1 Rare2

In [10]:
true_label = np.array(cell_names)
ip_file = '/fs/ess/scratch/PCON0022/mtduan/cell_line/'
filetype_name = 'Ordinary1_Rare2'

for file_num in tqdm(range(50)):
    l = []
    l = l + random.sample(ordinary_index,280)
    l = l + random.sample(two_rare_index_1,10)
    l = l + random.sample(two_rare_index_2,10)
    l = np.sort(np.array(l))
    label = true_label[l]
#     print(l)
#     print(label)
    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X

    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    ## python_input
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label300.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [10:31<00:00, 12.63s/it]
